In [4]:
import numpy as np 
import pandas as pd 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from random import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
data = pd.read_csv('train.tsv', sep='\t')

In [6]:
data.head(20)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [7]:
data = data.reindex(np.random.permutation(data.index))
#shuffle(data).head()

In [8]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
115596,115597,6160,mostly wordless ethnographic extras,1
45825,45826,2229,The film boasts at least a few good ideas and ...,3
111326,111327,5903,compelling than the circumstances of its making,2
11537,11538,497,About Schmidt,2
153562,153563,8386,the plot grinds itself out in increasingly inc...,1


In [9]:
data.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [10]:
Sentiments = to_categorical(data['Sentiment'], num_classes=5)

In [11]:
Sentiments[11]

array([1., 0., 0., 0., 0.], dtype=float32)

In [12]:
n_most_common_words = 20000
max_len = 49
tokenizer = Tokenizer(num_words=n_most_common_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Phrase'].values)
sequences = tokenizer.texts_to_sequences(data['Phrase'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#print(sequences)
#print(word_index)
X = pad_sequences(sequences)
print(X)

Found 15288 unique tokens.


[[    0     0     0 ... 10117 10118 10119]
 [    0     0     0 ...    65   709   152]
 [    0     0     0 ...     3    14   228]
 ...
 [    0     0     0 ...     0  1040  1709]
 [    0     0     0 ...     0     0  5219]
 [    0     0     0 ...   801     1    15]]


In [100]:
len(X[1290])

49

In [101]:
sequences[9]

[206, 282]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X , Sentiments, test_size=0.25, random_state=42)

In [14]:
y_test[0]

array([0., 0., 1., 0., 0.], dtype=float32)

In [15]:
epochs = 2
emb_dim = 64
batch_size = 64
#labels[:2]

In [16]:
model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))#to prevent the overfitting in the model
model.add(Dense(64,activation='relu'))
model.add(LSTM(64))
model.add(Dense(5, activation='softmax'))#activation function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])#adam adjusts the learning rate and learning rate calculates how fast weight will calculate 
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 49, 64)            1280000   
_________________________________________________________________
dense_1 (Dense)              (None, 49, 64)            4160      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 1,317,509
Trainable params: 1,317,509
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
history = model.fit(X_train, y_train, epochs=2, batch_size=batch_size,validation_split=0.2)

Instructions for updating:
Use tf.cast instead.


Train on 93636 samples, validate on 23409 samples
Epoch 1/2


   64/93636 [..............................] - ETA: 30:42 - loss: 1.6116 - acc: 0.0938

  128/93636 [..............................] - ETA: 17:50 - loss: 1.6089 - acc: 0.2578

  192/93636 [..............................] - ETA: 13:33 - loss: 1.6042 - acc: 0.3385

...

 4160/93636 [>.............................] - ETA: 5:13 - loss: 1.3085 - acc: 0.5091

 4224/93636 [>.............................] - ETA: 5:14 - loss: 1.3104 - acc: 0.5083

 4288/93636 [>.............................] - ETA: 5:13 - loss: 1.3103 - acc: 0.5072

 4352/93636 [>.............................] - ETA: 5:12 - loss: 1.3097 - acc: 0.5078

 4416/93636 [>.............................] - ETA: 5:11 - loss: 1.3090 - acc: 0.5070

In [18]:
accr = model.evaluate(X_test,y_test)

   32/39015 [..............................] - ETA: 1:05

   96/39015 [..............................] - ETA: 59s 

  192/39015 [..............................] - ETA: 49s

  288/39015 [..............................] - ETA: 46s

  384/39015 [..............................] - ETA: 44s

...

 9344/39015 [======>.......................] - ETA: 31s

 9472/39015 [======>.......................] - ETA: 31s

 9536/39015 [======>.......................] - ETA: 31s

 9632/39015 [======>.......................] - ETA: 30s

 9728/39015 [======>.......................] - ETA: 30s

In [19]:
print('Test set\n  Loss: {:f}\n  Accuracy: {:f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.829854
  Accuracy: 0.662668


In [23]:
txt = ["this is very,amazing,excellent, good"]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
sentiment_list = [0,1,2,3,4]
print(np.argmax(pred))
print(pred, sentiment_list[np.argmax(pred)])
print(pred)

4
[[6.4033520e-05 1.0741857e-04 2.3329575e-03 1.5843785e-01 8.3905774e-01]] 4
[[6.4033520e-05 1.0741857e-04 2.3329575e-03 1.5843785e-01 8.3905774e-01]]


In [24]:
test_data = pd.read_csv('test.tsv', sep='\t')

In [25]:
test_data.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [26]:
#phrase_id = []
sent_list = []
sentiment_list = [0,1,2,3,4]
for i in range(len(test_data)):
    txt = [test_data['Phrase'][i]]
    seq = tokenizer.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded)
    sent_list.append(sentiment_list[np.argmax(pred)])
    #phrase_id.append(test_data['PhraseId'][i])
    #print(pred, sentiment_list[np.argmax(pred)])

In [50]:
final_dataframe = pd.DataFrame()

In [51]:

final_dataframe['PhraseId'] = test_data['PhraseID']
final_dataframe['Sentiment'] = sent_list
final_dataframe['Phrase']=test_data['Phrase']

In [52]:
#final_dataframe.to_csv('submission.csv')

In [53]:

'''sent_list = []
sentiment_list = [0,1,2,3,4]

seq=tokenizer.texts_to_sequences(test_data["Phrase"].values)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
pred
for i in pred:
    sent_list.append(sentiment_list[np.argmax(i)])'''




In [54]:
f_df=pd.DataFrame()
f_df['PhraseId']=test_data['PhraseId']
f_df['sentiment']=sent_list
f_df['Phrase']=test_data['Phrase']

In [55]:
f_df.head(10)

,PhraseId,sentiment,Phrase
0,156061,2,An intermittently pleasing but mostly routine ...
1,156062,2,An intermittently pleasing but mostly routine ...
2,156063,2,An
3,156064,2,intermittently pleasing but mostly routine effort
4,156065,2,intermittently pleasing but mostly routine
5,156066,2,intermittently pleasing but
6,156067,2,intermittently pleasing
7,156068,2,intermittently
8,156069,3,pleasing
9,156070,2,but


In [56]:
len(f_df)

66292

In [57]:
type(pred[0])


numpy.ndarray